<img src="../images/cardboard.jpg">

## Making a profit off eBay: When and a little bit of How


Michael's mother is an avid eBay user. She sells all kinds of items on there - from clothing, to jewelry, and to most recently an old cell phone - but Curren, Charles, and he weren't so sure that that was the best way to make money while getting rid of goods you don't want. We decided to pursue looking into just whether or not eBay was a the right path to profitability. 

eBay has an extensive collection of information about all their auction listings. From time of listing and final price, to the feedback rating that sellers and buyers receive. We figured there was a way to consolidate and study this data to find something worth while. Considering Michael's mother recently sold a phone on there, we figured iPhones would be a good case study to look at. Many vendors now allow for trade-ins or store credit for devices one curently has, so we wanted to figure out when it was better to sell on eBay in comparison to selling back to a vendor. Below is a table of contents:


#Table of Contents
* [P1.Data](#P1.-Data)

## P1.Data

In [5]:
%matplotlib inline
import urllib, time, json, csv
import numpy as np
import pandas as pd

####Keys to make api calls from eBay.

In [4]:

#api_key = 'CharlesZ-11ce-4517-8df3-becc92494006'
#api_key = 'MichaelA-757e-4b18-8eeb-2a0a915831f3'
api_key = 'CurrenIy-ae09-43bf-83ac-d3cd5e2e02d1'

####The following three functions are what we are using to make api calls. These calls are through a url link, which we are using the library urllib to get the response data from. This response data is in the form of a json, which we convert to a dictionary using the json library.

In [5]:
# for finding api, findCompletedItems calls
def finding_data(page):
    # parameter and values for call
    paramKeys = ['OPERATION-NAME', 'SERVICE-VERSION','SECURITY-APPNAME','RESPONSE-DATA-FORMAT','categoryId', 'paginationInput.pageNumber', "keywords"]
    paramVals = ['findCompletedItems','1.7.0',api_key,'JSON&REST-PAYLOAD','9355', str(page), "iphone+5s+32GB"]

    # call url
    url_finding = 'http://svcs.ebay.com/services/search/FindingService/v1?'
    
    # constructing the url
    url = url_finding
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
    
    # loading the return response
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    # parse data for selected auctions
    return data['findCompletedItemsResponse'][0]['searchResult'][0]['item']



In [6]:
# for shopping api, getSingleItem calls
def item_data(item_id):
    # parameter and values for call
    paramKeys = ['callname', 'responseencoding', 'appid', 'siteid', 'version', 'ItemID', 'IncludeSelector']
    paramVals = ['GetSingleItem', 'JSON', api_key, '0', '515', item_id, 'Details']
    
    # call url
    url_item = 'http://open.api.ebay.com/shopping?'
    
    # constructing the url
    url = url_item
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
        
    # loading the return response, keep trying until success
    while True:
        try:
            response = urllib.urlopen(url)
            data = json.loads(response.read())
            break
        except ValueError:
            #time.sleep(1)
            pass
        
    
    #parse data for selected auction
    return data['Item']


In [9]:
# for shopping api, getUserProfile calls
def user_data(user_id):
    # parameter and values for call
    paramKeys = ['callname', 'responseencoding', 'appid', 'sideid', 'version', 'UserID', 'IncludeSelector']
    paramVals = ['GetUserProfile', 'JSON', api_key, '0', '525', user_id, 'FeedbackHistory']
    
    # call url
    url_user = 'http://open.api.ebay.com/shopping?'
    
    # constructing the url
    url = url_user
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
    # loading the return response, keep trying until success
    while True:
        try:
            response = urllib.urlopen(url)
            data = json.loads(response.read())
            break
        except ValueError:
            #time.sleep(1)
            pass
    
    #parse data for selected auction
    return data

####Here we are grabbing data from eBay's Finding API. The maximum number of calls we can make is 10000, but we also need to filter out duplicates (which sometimes occur). 

In [10]:
# finding api
# loop backwards to avoid conflicts
%time
repeat_set = set()
nodup_totaldata = []
for i in range(0,100):
    data = finding_data(100-i)
    time.sleep(.3) 
    for auc in data:
        if auc['itemId'][0] not in repeat_set:
            repeat_set.add(auc['itemId'][0])
            nodup_totaldata.append(auc)
    


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [11]:
repeat_set = set()
for auc in nodup_totaldata:
    repeat_set.add(auc['itemId'][0])
print len(repeat_set), len(nodup_totaldata)

9616 9616


####Create a new file and dump this data into it.

In [12]:
with open('completedItems.json', 'w') as outfile:
    json.dump(nodup_totaldata, outfile)

####Calling getSingleItem from eBay's Shopping API, this function takes some time because we are making over 9000 calls, and sometimes the calls fail, which means we need to remake the call.

In [14]:
%%time
auc_data = []
removed_set = set()
counter = 0
for auc in list(repeat_set):
    try:
        data = item_data(auc)
        auc_data.append(data)
        counter +=1
        print counter, "auctions retrieved"
    except KeyError:
        # auction id doesn't exist? remove auction from set
        removed_set.add(auc)
        repeat_set.remove(auc)
        #continue
        continue
    
    


1 auctions retrieved
2 auctions retrieved
3 auctions retrieved
4 auctions retrieved
5 auctions retrieved
6 auctions retrieved
7 auctions retrieved
8 auctions retrieved
9 auctions retrieved
10 auctions retrieved
11 auctions retrieved
12 auctions retrieved
13 auctions retrieved
14 auctions retrieved
15 auctions retrieved
16 auctions retrieved
17 auctions retrieved
18 auctions retrieved
19 auctions retrieved
20 auctions retrieved
21 auctions retrieved
22 auctions retrieved
23 auctions retrieved
24 auctions retrieved
25 auctions retrieved
26 auctions retrieved
27 auctions retrieved
28 auctions retrieved
29 auctions retrieved
30 auctions retrieved
31 auctions retrieved
32 auctions retrieved
33 auctions retrieved
34 auctions retrieved
35 auctions retrieved
36 auctions retrieved
37 auctions retrieved
38 auctions retrieved
39 auctions retrieved
40 auctions retrieved
41 auctions retrieved
42 auctions retrieved
43 auctions retrieved
44 auctions retrieved
45 auctions retrieved
46 auctions retriev

In [15]:
with open('getSingleItem.json', 'w') as outfile:
    json.dump(auc_data,outfile)


#### We get the set of unique users from all this data, to use for the final call to get the user profile information

In [16]:
user_ids = set()
for x in auc_data:
    user_ids.add(x['Seller']['UserID'])
print len(user_ids)    


5575


In [17]:
list(user_ids)[0]

u'reuv_litz'

#### Here we are making the calls to get user profile information.

In [18]:
%%time
user_info = []
counter = 0
for user_id in list(user_ids):
    try:
        data = user_data(user_id)
        user_info.append(data)
        counter +=1
        print counter, "users retrieved"
    except KeyError:
        # auction id doesn't exist? remove auction from set
        print "cannot find user", user_id
        #continue
        continue
    
        
    


1 users retrieved
2 users retrieved
3 users retrieved
4 users retrieved
5 users retrieved
6 users retrieved
7 users retrieved
8 users retrieved
9 users retrieved
10 users retrieved
11 users retrieved
12 users retrieved
13 users retrieved
14 users retrieved
15 users retrieved
16 users retrieved
17 users retrieved
18 users retrieved
19 users retrieved
20 users retrieved
21 users retrieved
22 users retrieved
23 users retrieved
24 users retrieved
25 users retrieved
26 users retrieved
27 users retrieved
28 users retrieved
29 users retrieved
30 users retrieved
31 users retrieved
32 users retrieved
33 users retrieved
34 users retrieved
35 users retrieved
36 users retrieved
37 users retrieved
38 users retrieved
39 users retrieved
40 users retrieved
41 users retrieved
42 users retrieved
43 users retrieved
44 users retrieved
45 users retrieved
46 users retrieved
47 users retrieved
48 users retrieved
49 users retrieved
50 users retrieved
51 users retrieved
52 users retrieved
53 users retrieved
54

In [19]:
counter = 0

for auc in nodup_totaldata:
    if auc['sellingStatus'][0]['sellingState'][0] == 'EndedWithSales':
        counter +=1
print counter

6414


#### Dumping our data into a final json.

In [20]:
with open('getUserProfile.json', 'w') as fd:
    json.dump(user_info, fd)

#### The difference in the amount of data between the two jsons makes sense because some auctions were unable to be found with our second API call, getSingleItem. This is a problem on eBay's side, so we are unable to fix it.

In [21]:
len(nodup_totaldata), len(auc_data)

(9616, 9571)

#### Last checks to make sure that the data is consistent throughout. The code below just checks whether all of the itemIds we've gathered in the first json are the same as the ones we've gathered in the second json.

In [23]:
json1 = open('completedItems.json', 'r')
json2 = open('getSingleItem.json', 'r')

doc1 = json.load(json1)
doc2 = json.load(json2)


In [24]:
newItemIdSet = set()
counter = 0
for auc in doc1:
    newItemIdSet.add(auc['itemId'][0])
for auc in doc2:
    if auc['ItemID'] not in newItemIdSet:
        counter += 1
print counter

0


In [25]:
json1.close()
json2.close()